<a href="https://colab.research.google.com/github/Bhairavi-shah/AbstractiveUserReviewConsolidation/blob/master/Module2.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
%%time
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
pd.set_option('display.max_colwidth',1000)

df = pd.read_csv(F"/content/gdrive/My Drive/PROJECT | S7-S8/Data/training.csv", engine='python', names=["target", "ids", "date", "flag", "user", "text"])
df = df[:1000].append(df[-1000:])
print(df)
print(df.columns)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
         target  ...                                                                                                                 text
0             0  ...  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
1             0  ...      is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
2             0  ...                            @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
3             0  ...                                                                      my whole body feels itchy and like its on fire 
4             0  ...      @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
...         ...  ...                     

In [6]:
from bs4 import BeautifulSoup
import urllib3
import json
http=urllib3.PoolManager()
Abbr_dict={}

#Function to get the Slangs from https://www.noslang.com/dictionary/
def getAbbr(alpha):
    global Abbr_dict
    r=http.request('GET','https://www.noslang.com/dictionary/'+alpha)
    soup=BeautifulSoup(r.data,'html.parser')
    
    for i in soup.findAll('div',{'class':'dictionary-word'}): 

        abbr=i.find('abbr')['title']
        Abbr_dict[i.find('span').text[:-2]]=abbr

#Generating a-z
linkDict=[]
for x in range(97,123):
    linkDict.append(chr(x))

#Creating Links for https://www.noslang.com/dictionary/a...https://www.noslang.com/dictionary/b....etc
for i in linkDict:
    getAbbr(i)

print(Abbr_dict)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

{'a$$': 'ass', 'a&f': 'always and forever', "a'ight": 'alright', 'a.i.m.': 'aol instant messanger', 'a/l': 'age and location', 'a/m': 'away message', 'a/s/l': 'age,sex,location', 'a/s/l/p': 'age/sex/location/picture', 'a/s/l/r': 'age, sex, location, race', 'a1t': 'anyone there', 'A2A': 'Ask to Answer', 'a3': 'anyplace, anywhere, anytime', 'a4u': 'all for you', 'A7X': 'Avenged Sevenfold', 'aaaaa': 'American Assosciation Against Acronym Abuse', 'aabf': 'as a best friend', 'aaf': 'as a friend', 'aak': 'Alive and Kicking', 'aamof': 'as a matter of fact', 'aatf': 'always and totally forever', 'aatw': 'all around the world', 'abd': 'Already Been Done', 'abend': 'absent by enforced net deprivation', 'abft': 'About f**king Time', 'aboot': 'about', 'abreev': 'abbreviation', 'absnt': 'absent', 'abt': 'about', 'abwt': 'about', 'acc': 'account', 'acct': 'account', 'acgaf': "Absolutely couldn't give a f**k", 'ack': 'acknowledged', 'addy': 'address', 'adhd': 'Attention Deficit Hyperactivity Disorder

In [72]:
%%time
from bs4 import BeautifulSoup
import re
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("wordnet")
stop_words = list(STOP_WORDS)
wordnet_lemmatizer = WordNetLemmatizer()

def text_cleaner(strg):
    newString = strg.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([Abbr_dict[t] if t in Abbr_dict else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    newString = re.sub('http|https', "", newString)
    tokens = []
    for w in newString.split():
      if (w not in stop_words) and (w not in "?:!.,;"):
        if w in Abbr_dict.keys() and len(Abbr_dict[w])<2 :
          tokens += [wordnet_lemmatizer.lemmatize(Abbr_dict[w])]
        elif w in Abbr_dict.keys():
          tokens += text_cleaner(Abbr_dict[w]).split()
        else :
          tokens += [wordnet_lemmatizer.lemmatize(w)]
    return (" ".join(tokens)).strip()

df['PreprocessedText'] = df.text.apply(lambda x: text_cleaner(x))
print(df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
         target  ...                                                    PreprocessedText
0             0  ...  switchfoot twitpic com zl awww bummer shoulda got david carr day d
1             0  ...        upset t update facebook texting cry result school today blah
2             0  ...                    kenichan dived time ball managed save rest bound
3             0  ...                                           body feel itchy like fire
4             0  ...                                      nationwideclass behaving mad t
...         ...  ...                                                                 ...
1599995       4  ...                                     woke having school best feeling
1599996       4  ...            thewdb com cool hear old walt 

In [73]:
%%time
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import nltk
nltk.download('punkt')

# Creating the Bag of Words model 
word2count = {}
for data in df['PreprocessedText'].values: 
    words = nltk.word_tokenize(data) 
    for word in words: 
        if word not in word2count.keys(): 
            word2count[word] = 1
        else: 
            word2count[word] += 1

print(word2count["like"], word2count["hate"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
98 34
CPU times: user 195 ms, sys: 0 ns, total: 195 ms
Wall time: 196 ms


In [74]:
%%time
import heapq
freq_words = heapq.nlargest(500, word2count, key=word2count.get)
print(freq_words)

['t', 'good', 'day', 'normal', 'denial', 'love', 'going', 'like', 'time', 'know', 'want', 'today', 'work', 'laughing', 'oh', 'got', 'com', 'loud', 'thanks', 'yeah', 'new', 'Instant', 'Message', 'think', 'sad', 'morning', 'night', 'sleep', 've', 'tomorrow', 'feel', 'miss', 'hope', 'home', 'great', 'need', 'sorry', 'll', 'okay', 'people', 'getting', 'didn', 'world', 'twitter', 'school', 'watching', 'thing', 'yes', 'wish', 'hate', 'haha', 'bad', 'week', 'sick', 'hour', 'better', 'way', 'twitpic', 'come', 'as', 'right', 'fun', 'd', 'happy', 'lot', 'tonight', 'friend', 'tweet', 'cool', 'wait', 'hear', 'best', 'year', 'life', 'pretty', 'hey', 'bed', 'sure', 'nice', 'early', 'find', 'picture', 'old', 'job', 'look', 'web', 'house', 'th', 'watch', 'won', 'start', 'working', 'mom', 'video', 'ready', 'bit', 'lost', 'soon', 'little', 'cant', 'tired', 'thank', 'looking', 'god', 'cry', 'feeling', 'girl', 'hard', 'movie', 'coming', 'listening', 'long', 'thought', 'missed', 'yay', 'having', 'wide', 'k

In [75]:
%%time
X = [] 
i = 0
for data in df['PreprocessedText'].values: 
    vector = []
    for word in freq_words: 
        if word in nltk.word_tokenize(data): 
            vector.append(1) 
        else: 
            vector.append(0) 
    i+=1
    print(i)
    X.append(vector)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [76]:
Xdf = pd.DataFrame(X, columns = freq_words)
print(Xdf)

      t  good  day  normal  denial  love  ...  smile  blip  mad  nope  user  loss
0     0     0    1       0       0     0  ...      0     0    0     0     0     0
1     1     0    0       0       0     0  ...      0     0    0     0     0     0
2     0     0    0       0       0     0  ...      0     0    0     0     0     0
3     0     0    0       0       0     0  ...      0     0    0     0     0     0
4     1     0    0       0       0     0  ...      0     0    1     0     0     0
...  ..   ...  ...     ...     ...   ...  ...    ...   ...  ...   ...   ...   ...
1995  0     0    0       0       0     0  ...      0     0    0     0     0     0
1996  0     0    0       0       0     0  ...      0     1    0     0     0     0
1997  0     0    0       0       0     0  ...      0     0    0     0     0     0
1998  0     0    0       0       0     0  ...      0     0    0     0     0     0
1999  0     0    0       0       0     0  ...      0     0    0     0     0     0

[2000 rows x 50

In [77]:
df_traintest = Xdf
print(df_traintest)
y = df['target']
print(y)

      t  good  day  normal  denial  love  ...  smile  blip  mad  nope  user  loss
0     0     0    1       0       0     0  ...      0     0    0     0     0     0
1     1     0    0       0       0     0  ...      0     0    0     0     0     0
2     0     0    0       0       0     0  ...      0     0    0     0     0     0
3     0     0    0       0       0     0  ...      0     0    0     0     0     0
4     1     0    0       0       0     0  ...      0     0    1     0     0     0
...  ..   ...  ...     ...     ...   ...  ...    ...   ...  ...   ...   ...   ...
1995  0     0    0       0       0     0  ...      0     0    0     0     0     0
1996  0     0    0       0       0     0  ...      0     1    0     0     0     0
1997  0     0    0       0       0     0  ...      0     0    0     0     0     0
1998  0     0    0       0       0     0  ...      0     0    0     0     0     0
1999  0     0    0       0       0     0  ...      0     0    0     0     0     0

[2000 rows x 50

In [78]:
%%time
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(df_traintest, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1600, 500) (1600,)
(400, 500) (400,)
CPU times: user 8.45 ms, sys: 1 ms, total: 9.45 ms
Wall time: 8.79 ms


In [79]:
%%time
#fit a model
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
random_forest.fit(X_train, y_train.values.ravel())

predictions = random_forest.predict(X_test)
print(predictions)

print("Score:", random_forest.score(X_test, y_test))

[4 4 4 4 0 4 0 4 0 0 4 4 0 4 4 0 4 4 4 0 0 4 0 4 0 4 4 0 0 0 4 4 4 4 0 0 4
 4 0 0 0 4 4 0 0 0 4 4 4 0 4 4 4 4 0 4 0 0 0 4 0 0 4 4 0 4 4 0 0 4 0 4 0 4
 0 4 0 4 0 4 0 4 4 0 0 4 0 4 0 4 4 0 0 0 4 4 0 4 4 0 0 4 4 4 0 0 0 0 0 4 0
 4 4 0 4 0 0 0 4 0 4 4 0 0 4 4 4 0 4 0 0 4 4 0 4 4 0 0 0 4 0 0 4 0 4 0 0 0
 0 4 0 4 0 4 0 0 4 4 0 4 0 4 0 4 4 4 0 4 0 4 4 0 4 4 4 4 0 4 0 4 4 4 0 4 0
 0 0 4 4 0 4 0 4 4 4 0 0 0 0 0 4 0 4 0 0 0 4 4 0 4 4 4 0 4 0 4 4 4 0 0 0 4
 4 0 0 0 0 4 0 4 0 4 4 0 0 4 4 4 4 0 4 4 4 4 4 0 4 4 4 4 4 4 0 4 0 4 0 4 4
 4 0 4 0 0 4 4 4 4 4 0 0 4 0 4 4 4 4 4 4 0 0 4 4 0 0 0 0 4 0 0 0 4 0 0 0 0
 4 0 0 4 4 4 0 0 4 0 4 0 4 4 0 4 0 4 0 0 0 0 4 0 0 4 4 4 4 0 4 0 0 0 4 0 4
 0 0 4 0 4 4 4 0 0 0 0 0 0 0 0 4 0 0 0 4 4 0 0 0 4 0 0 4 0 4 4 4 0 4 0 4 0
 4 4 4 0 4 4 0 4 0 0 0 4 0 4 4 0 0 4 4 4 4 4 0 0 0 4 0 0 0 0]
Score: 0.685
CPU times: user 953 ms, sys: 3 ms, total: 956 ms
Wall time: 956 ms


In [0]:
import pickle
filename = 'random_forest.pkl'
pickle.dump(random_forest, open(filename, 'wb'))

In [81]:
%%time
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train.values.ravel())
print("Score DecisionTree            :", dt.score(X_test, y_test.values.ravel()))

Score DecisionTree            : 0.6425
CPU times: user 179 ms, sys: 1.01 ms, total: 180 ms
Wall time: 179 ms


In [0]:
filename = 'decision_tree.pkl'
pickle.dump(dt, open(filename, 'wb'))

In [83]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train.values.ravel())
print("Score KNeighbors            :", knn.score(X_test, y_test.values.ravel()))

Score KNeighbors            : 0.6025
CPU times: user 696 ms, sys: 991 µs, total: 697 ms
Wall time: 699 ms


In [0]:
filename = 'k_neighbours.pkl'
pickle.dump(knn, open(filename, 'wb'))

In [85]:
%%time
from sklearn import svm
supportvm = svm.SVC() 
supportvm.fit(X_train, y_train.values.ravel())
print("Score Support Vector Machine:", supportvm.score(X_test, y_test.values.ravel()))

Score Support Vector Machine: 0.725
CPU times: user 2.38 s, sys: 2.94 ms, total: 2.38 s
Wall time: 2.4 s


In [0]:
filename = 'SVM.pkl'
pickle.dump(supportvm, open(filename, 'wb'))

In [87]:
%%time
df = pd.read_json(r"/content/gdrive/My Drive/PROJECT | S7-S8/Data/AMAZON_FASHION_5.json", lines = True)
df.groupby('asin').mean()
df_test = (df.loc[df['asin'] == 'B0014F7B98'])
print(df_test.head())
print(df_test.columns)
df_test["PreprocessedReviews"] = df_test.reviewText.apply(lambda x: text_cleaner(str(x)))
print(df_test)

X = [] 
for data in df_test['PreprocessedReviews'].values: 
    vector = []
    for word in freq_words: 
        if word in nltk.word_tokenize(data): 
            vector.append(1) 
        else: 
            vector.append(0) 
    X.append(vector)

Xdf = pd.DataFrame(X, columns = freq_words)
print(Xdf)

     overall  verified   reviewTime  ... unixReviewTime  vote image
869        5      True  03 20, 2016  ...     1458432000   NaN   NaN
870        5      True  03 18, 2016  ...     1458259200   NaN   NaN
871        4      True  03 16, 2016  ...     1458086400   NaN   NaN
872        5      True  03 13, 2016  ...     1457827200  22.0   NaN
873        5      True   03 6, 2016  ...     1457222400   NaN   NaN

[5 rows x 12 columns]
Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image'],
      dtype='object')
      overall  ...                                                                                                                                                                                                                       PreprocessedReviews
869         5  ...                                                                                                              

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


     t  good  day  normal  denial  love  ...  smile  blip  mad  nope  user  loss
0    0     0    0       0       0     1  ...      0     0    0     0     0     0
1    0     0    0       0       0     0  ...      0     0    0     0     0     0
2    0     0    0       0       0     1  ...      0     0    0     0     0     0
3    1     1    0       1       1     0  ...      0     0    0     0     0     0
4    0     0    0       0       0     0  ...      0     0    0     0     0     0
..  ..   ...  ...     ...     ...   ...  ...    ...   ...  ...   ...   ...   ...
369  0     0    1       0       0     0  ...      0     0    0     0     0     0
370  0     0    0       0       0     1  ...      0     0    0     0     0     0
371  0     0    0       0       0     0  ...      0     0    0     0     0     0
372  0     0    0       0       0     0  ...      0     0    0     0     0     0
373  0     0    0       0       0     0  ...      0     0    0     0     0     0

[374 rows x 500 columns]
CP

In [88]:
print(random_forest.predict(Xdf))
print(dt.predict(Xdf))
print(knn.predict(Xdf))
print(supportvm.predict(Xdf))

[4 4 4 4 4 4 4 4 0 4 0 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 0 4
 4 4 4 4 4 0 0 0 4 4 4 4 4 0 4 4 4 4 0 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4
 4 0 4 0 4 0 4 0 4 4 0 4 4 4 4 0 4 4 4 0 4 4 0 4 4 0 4 0 0 4 4 4 4 4 4 4 4
 0 4 4 0 4 0 4 4 0 4 4 0 4 4 0 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 0 0 4 4 4 0
 4 4 4 4 4 0 0 4 0 4 4 4 4 4 4 4 0 0 4 4 4 4 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 0 0 4 0 4 4 4 4 4 4 4 4 4 4 4 0 0 4 4 0 4 4 4 4 0 0 4 4 4 0 4 4 0
 0 4 4 4 4 4 4 0 4 4 0 4 4 4 4 4 4 4 4 4 0 4 4 4 4 0 0 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 0 0 0 4 0 4 0 0 4 4 4 4 4 4 0 4 0 4 0 4 4 4 4 0 0 0 4 4 4 4 4
 4 4 0 0 4 4 4 4 4 4 4 0 4 0 4 4 4 0 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 0 0 4 4 4 4 4 4 4 4 4 0 4 4 0 4 0 4 4 4 4 0 0 0 0 4 4 0 4 0
 4 4 4 0]
[4 4 4 4 4 4 4 4 0 4 0 4 4 4 0 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 0 0 4
 4 4 4 4 4 4 0 0 4 4 4 4 4 0 4 4 4 4 4 4 4 0 0 4 4 4 4 0 4 4 4 4 4 4 0 4 4
 4 0 4 0 4 0 4 0 4 4 4 4 4 4 4 0 4 4 4 0 4 4 4 4 4 4 4 4 0 4 4 4 4 4 0 4 4
 0 4 4 4 4 0 4 